In [6]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [11]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート_2')
# データフレームを表示して確認
print(df)



                                               アンケート
0  「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...
1  「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...
2  「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...
3  「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...
4  「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...
5              「使ってみたが髪がギシギシになり、残念ながら私の髪質には会いませんでした」
6        「ひどい商品！SNSの広告に流れてきてみんないいって言ってのに！もう二度と使いません」
7  「可もなく不可もなく。期待値が高すぎたのかも。でもどうしてこんなに流行ってるのでしょうか？特...


In [10]:
# アンケートの主要キーワードをLLMで抽出する関数
def extract_keywords(text):
    # 2. データをLLM用にテキスト形式に変換
    # プロンプトの作成
    prompt_text = f"以下のアンケートから主要なキーワードをカンマ区切りで抽出してください。\nアンケート: {text}"

    # 3. OpenAI APIの呼び出し
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # キーワード抽出結果を取得
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [4]:
# テストコードで確認
text = extract_keywords("pandasは、Excelファイル（`.xls`、`.xlsx` 形式）の読み書きに加え、データの集計や変換、フィルタリングといった複雑な操作を手軽に行えるライブラリです。")
print(text)

pandas, Excelファイル, 読み書き, データ, 集計, 変換, フィルタリング, 複雑な操作, ライブラリ


In [12]:
# 4. 分析結果をデータフレームに変換

# B列（キーワード）を初期化
df['キーワード'] = ""

# for文で各アンケート行に対してキーワード抽出を実行
for index, row in df.iterrows():
    # A列のアンケート内容を取得
    survey_text = row['アンケート']
    
    # キーワード抽出関数を呼び出し、結果をB列に保存
    keywords = extract_keywords(survey_text)
    df.at[index, 'キーワード'] = keywords

# データフレームを表示して確認
print(df)

                                               アンケート  \
0  「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...   
1  「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...   
2  「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...   
3  「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...   
4  「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...   
5              「使ってみたが髪がギシギシになり、残念ながら私の髪質には会いませんでした」   
6        「ひどい商品！SNSの広告に流れてきてみんないいって言ってのに！もう二度と使いません」   
7  「可もなく不可もなく。期待値が高すぎたのかも。でもどうしてこんなに流行ってるのでしょうか？特...   

                                               キーワード  
0  髪, しっとり, サラサラ, 驚き, 香り, リラックス, 泡立ち, 少量, 洗える, コス...  
1  洗い上がり, スッキリ, 髪, きしまず, 指通り, 乾燥, 髪質, シャンプー, まとまり...  
2    香り, 甘め, 好み, 強く感じる, 洗い上がり, 軽やか, 頭皮, すっきり, 夏場, 快適  
3  ボトルデザイン, シンプル, おしゃれ, バスルーム, 使用感, シャンプー, 乾燥, パサ...  
4  頭皮のかゆみ, 敏感肌, シャンプー, 刺激が少ない, 使いやすい, 香り, ナチュラル, ...  
5                              髪, ギシギシ, 残念, 髪質, 合わない  
6                          ひどい商品, SNS, 広告, いい, 使いません  
7               可もなく不可もなく, 期待値, 流行, 特筆すべきもの, 見当たりません  


In [6]:
# 5. 結果をExcelファイルに保存
df.to_excel("アンケート分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

# A列の各行のアンケートに対してキーワード抽出を実行し、B列に書き込む
df['キーワード'] = df['アンケート'].apply(extract_keywords)

# 結果をExcelファイルに保存
df.to_excel("アンケート分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
